### Original Ethereum Transaction Data — Feature Dictionary

| Field | Type | Meaning | Use | Notes |
|---|---|---|---|---|
| hash | string | Unique transaction hash | context | Not modeled directly; can be used as row ID |
| nonce | int | Per-sender transaction count at time of tx | dropped | Not used in current feature set |
| transaction_index | int | Position of tx within block | dropped | Block-local ordering only |
| from_address | string | Sender address | analysis | Used as key to build per-address features |
| to_address | string | Recipient address | analysis | Used as key to build per-address features |
| value | float | Transferred ETH amount in wei | analysis | Aggregated into incoming/outgoing amount features |
| gas | int | Gas limit specified for tx | analysis | Used for avg gas limit per address |
| gas_price | float | Gas price offered (wei per gas unit) | analysis | Used for avg gas price per address |
| input | string | Hex calldata / payload | dropped | Not parsed in this project |
| receipt_cumulative_gas_used | int | Total gas used in block up to this tx | dropped | Not used in current features |
| receipt_gas_used | int | Gas used by this tx alone | dropped | Redundant with other gas behavior for this analysis |
| block_timestamp | string → datetime | Block time for tx | analysis | Parsed to UTC; basis for all time/sequence features |
| block_number | int | Block height containing tx | dropped | Highly collinear with timestamp; not modeled directly |
| block_hash | string | Hash of containing block | dropped | Not used in current features |
| from_scam | int (0/1) | Source is labeled scam address | analysis | Used to construct per-address Scam label |
| to_scam | int (0/1) | Destination is labeled scam address | analysis | Used to construct per-address Scam label |
| from_category | string | Labeled category for sender | analysis | Used to flag scam/fraud/phish categories |
| to_category | string | Labeled category for recipient | analysis | Used to flag scam/fraud/phish categories |

### Engineered Address-Level Feature Table — Feature Dictionary

Index: each row corresponds to a unique Ethereum `Address` (string), aggregated over all transactions.

| Field | Type | Meaning | Use | Notes |
|---|---|---|---|---|
| in_degree | int | Count of incoming txs to address | analysis | Number of rows where address is `to_address` |
| out_degree | int | Count of outgoing txs from address | analysis | Number of rows where address is `from_address` |
| unique in_degree | int | Number of distinct senders to this address | analysis | Unique `from_address` values seen as incoming |
| unique out_degree | int | Number of distinct recipients from this address | analysis | Unique `to_address` values seen as outgoing |
| Avg amount incoming | float | Mean incoming transfer value (wei) | analysis | Averaged over all txs where address is recipient |
| Total amount incoming | float | Sum of incoming transfer value (wei) | analysis | Total ETH in wei received |
| Max amount incoming | float | Maximum single incoming value (wei) | analysis | Largest inbound transfer |
| Min amount incoming | float | Minimum single incoming value (wei) | analysis | Smallest inbound transfer (0 if none) |
| Avg amount outgoing | float | Mean outgoing transfer value (wei) | analysis | Averaged over all txs sent by address |
| Total amount outgoing | float | Sum of outgoing transfer value (wei) | analysis | Total ETH in wei sent |
| Max amount outgoing | float | Maximum single outgoing value (wei) | analysis | Largest outbound transfer |
| Min amount outgoing | float | Minimum single outgoing value (wei) | analysis | Smallest outbound transfer (0 if none) |
| Avg time incoming | float | Mean timestamp of incoming txs (seconds) | analysis | Seconds since earliest block in dataset |
| Avg time outgoing | float | Mean timestamp of outgoing txs (seconds) | analysis | Seconds since earliest block in dataset |
| Total Tx Time | float | Sum of the actual time gaps between consecutive transactions (seconds) | analysis | For addresses with ≥2 txs, this equals the sum of all inter-transaction intervals; 0 if ≤1 tx |
| Active Duration | float | Lifespan between first and last tx (s) | analysis | 0 if only a single tx |
| Mean time interval | float | Mean gap between consecutive txs (s) | analysis | 0 if ≤1 tx |
| Max time interval | float | Largest gap between consecutive txs (s) | analysis | 0 if ≤1 tx |
| Min time interval | float | Smallest gap between consecutive txs (s) | analysis | 0 if ≤1 tx |
| Burstiness | float | max_gap / median_gap of tx times | analysis | 0 for ≤2 txs; higher = more bursty activity |
| Tx count | int | Total number of txs (in + out) for this address | analysis | Equals in_degree + out_degree over the dataset window |
| Activity Density | float | Tx count per second of Active Duration | analysis | `Tx count / (Active Duration + 1)` to avoid division by zero |
| Incoming count | int | Number of incoming txs | analysis | Count of records where address is recipient |
| Outgoing count | int | Number of outgoing txs | analysis | Count of records where address is sender |
| In/Out Ratio | float | (Incoming count + 1) divided by (Outgoing count + 1) | analysis | Higher values = sink-like behavior; +1 terms avoid divide-by-zero |
| Hour mean | float | Mean hour of day of activity (0–23) | analysis | Computed from UTC timestamps across all txs |
| Hour entropy | float | Entropy of hourly activity distribution (bits) | analysis | 0 = all txs at one hour; higher = spread across hours |
| Last seen | float | Timestamp of most recent tx (s) | analysis | Seconds since earliest block in dataset |
| Recency | float | How long before dataset end address was last active (s) | analysis | `dataset_end_ts_seconds − Last seen` |
| Avg gas price | float | Mean gas price used by address (wei per gas) | analysis | Aggregated across all in/out txs |
| Avg gas limit | float | Mean gas limit on txs involving address | analysis | Aggregated across all in/out txs |
| Scam | int (0/1) | Address labeled as scam-related | target | Derived from from_scam/to_scam and *_category text |